## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Открытие файла с данными и изучение общей информации. 

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод
Данные о заемщиках содержат 21525 записей, значения которых распределены по 12 столбцам, которые отражают такие параметры как наличие детей, стаж работы, возраст, образование, семейный статус, пол, сведения о доходах и наличии долговой нагрузки а также цель получения кредита.

В столбцах 'days_employed' и 'total_income' содержатся пропуски.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
mask = df['days_employed'].isnull() #отфильтруем строки с пропущенными значениями
df[mask].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [5]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [6]:
#кол-во уникальных значений столбца 'days employed'
len(df['days_employed'].unique())

19352

In [7]:
#кол-во уникальных значений столбца 'total_income'
len(df['total_income'].unique())

19352

In [8]:
mask1 = df['total_income'].isnull()
df_miss = df[mask1]
df.head(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья


In [9]:
df_miss['income_type'].value_counts(normalize = True)

сотрудник          0.508280
компаньон          0.233671
пенсионер          0.189972
госслужащий        0.067617
предприниматель    0.000460
Name: income_type, dtype: float64

Большинство неуказавших доход не пенсионеры, а сотрудники неизвестных компаний

In [10]:
#сравним с первоначальными данными
df['income_type'].value_counts(normalize = True)

сотрудник          0.516562
компаньон          0.236237
пенсионер          0.179141
госслужащий        0.067782
предприниматель    0.000093
безработный        0.000093
студент            0.000046
в декрете          0.000046
Name: income_type, dtype: float64

Делаем вывод, что распределения типов занятости в данных с пропусками не отличается, то есть пропуски случайны.

In [11]:
total_income_mean = df['total_income'].mean()

In [12]:
#заполняем пропущенные значения столбца 'total_income' средним значением
df['total_income'] = df['total_income'].fillna(value = total_income_mean)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод
Обнаруженные пропущенные значения в столбце со стажем ('days_employed') были заменены на медиану, как более характерное значение для данного параметра.

Возможная причина появления пропущенных данных - человеческий фактор: при заполнении анкеты данные поля не были заполнены.

Пропущенные значения в столбце с доходом ('total_income') были заменены на среднее значение (различия между средним значением и медианой в данном случае было несущественным).

### Замена типа данных

In [14]:
df['total_income'] = df['total_income'].astype('int')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


### Вывод
Вещественный тип значений в столбце 'total_income' был заменен на целочисленный.

### Обработка дубликатов

In [16]:
#посмотрим количество дубликатов-строк
df.duplicated().sum()

54

In [17]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [18]:
df['education'] = df['education'].str.lower()

In [19]:
df.duplicated().sum()

71

После того как в столбце education все значения приведены к нижнему регистру, появились новые дубликаты.

In [20]:
df.drop_duplicates(inplace = True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int32  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод
Были выявлены и удалены 71 дубликат строк в таблице.

Для выявления строк-дубликатов к DataFrame был применен метод duplicated() и подсчитано количество методом sum().
Кроме того, значения столбца education были приведены к нижнему регистру, что выявило дополнительные дубликаты.

Для удаления строк-дублткатов к DataFrame был применен метод drop_duplicates() c параметром inplace = True для сохранения DataFrame без строк-дубликатов.

Возможная причина появления дубликатов: пользователями были несколько раз отправлены идентичные данные, пользователями были введены идентичные данные с использованием прописных и строчных букв.

### Лемматизация

In [22]:
#импортируем библиотеку PyMystem
from pymystem3 import Mystem
m = Mystem()

In [23]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [24]:
#преобразуем Series в string с пробелами в качестве разделителя
text = df['purpose']
text_string = text.str.cat(sep=' ')

In [25]:
lemmas = m.lemmatize(text_string)

In [26]:
#импортируем счетчик и посмотрим наиболее часто встречающиеся леммы
from collections import Counter 
c = Counter(lemmas)
c.most_common()

[(' ', 55023),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('подержать', 853),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('приобретение', 461),
 ('профильный', 436),
 ('подержанный', 111),
 ('\n', 1)]

In [27]:
#функция лемматизации текста
def lemma_purpose(text):
    lemmas = m.lemmatize(text)
    return lemmas

In [28]:
#проверка функции
lemma_purpose('кредит на покупку недвижимости')

['кредит', ' ', 'на', ' ', 'покупка', ' ', 'недвижимость', '\n']

In [29]:
#добавим столбец с леммами целей кредита
df['lemma_purpose'] = df['purpose'].apply(lemma_purpose)

KeyboardInterrupt: 

In [ ]:
df.head()

### Вывод
Лемматизация столбца с целью получения кредита показывает, что наиболее популярными целями являются:
* покупка недвижимости
* покупка автомобиля
* образование
* операция
* свадьба
* строительство

### Категоризация данных

In [ ]:
#функция возвращает категорию цели кредита в зависимости от встречающихся слов
def purpose_cat(data):
    if ('недвижимость' in data) or ('жилье' in data):
        return 'недвижимость'
    if 'автомобиль' in data:
        return 'авто'
    if 'образование' in data:
        return 'образование'
    if 'операция' in data:
        return 'операция'
    if 'свадьба' in data:
        return 'свадьба'
    if 'строительство' in data:
        return 'строительство'
    return 'другое'

#добавляем к таблице новый столбец с категориями целей кредита
df['purpose_category'] = df['lemma_purpose'].apply(purpose_cat)

In [ ]:
df.head(20)

In [ ]:
df['purpose_category'].value_counts().head(10)

### Вывод

Во-первых категорирование целей кредита с помощью лемм выявило то, что отдельных целей "операция" и "строительство" - нет, все цели укладываются в категории:
* недвижимость
* авто
* образование
* свадьба

Во-вторых, удивительно что категория "другое" отсутствует!

На данном этапе категоризированы цели кредита, для дальнейших ответов на вопросы будут категоризированы данные в столбцах с семейным положением, уровнем дохода.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
df['debt'].value_counts()

In [ ]:
df['debt'].mean()

In [ ]:
df['children'].value_counts()

In [ ]:
def child_order(data2):
    if data2 == 0:
        return 0
    elif data2 == 1:
        return 1
    elif data2 == 2:
        return 2
    elif data2 == 3:
        return 3
    elif data2 == 4:
        return 4
    return 0
df['children'] = df['children'].apply(child_order)

In [ ]:
df['children'].value_counts()

In [ ]:
df.groupby('children')['debt'].agg(['count','mean'])

In [ ]:
df.groupby('children')['debt'].mean().plot()

### Вывод

## У бездетных самая маленькая просрочка (7%), у клиентов с детьми незначительно выше процент просрочки (порядка 9,5%)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
df['family_status'].value_counts()

In [ ]:
def family_category(data3):
    if (data3 == 'женат / замужем') or (data3 == 'гражданский брак'):
        return 'семья'
    return 'нет семьи'
df['family_cat'] = df['family_status'].apply(family_category)

In [ ]:
df.head()

In [ ]:
df.groupby('family_status')['debt'].agg(['count','mean'])

In [ ]:
df['family_cat'].value_counts()

In [ ]:
df.groupby('family_cat')['debt'].agg(['count','mean'])

In [ ]:
def family_category2(data4):
    if (data4 == 'Не женат / не замужем'):
        return 'не состоял в семейных отношениях'
    return 'состоял/состоит в семейных отношениях'
df['family_cat2'] = df['family_status'].apply(family_category2)

In [ ]:
df.head()

In [ ]:
df.pivot_table(index='family_cat2',
              values=['debt'],
               aggfunc=['count','mean'])

### У тех кто состоял/состоит в семейных отношениях просрочек меньше на 2%

### Вывод

## У семейных людей меньше просрочек по кредитам, разница составляет около 1%, а также, у людей не состоявших в семейных отношениях на 2% больше просрочек

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
df['total_income'].sort_values()

In [ ]:
df['income_cat'] = pd.cut(df['total_income'],[20000, 70000, 100000, 150000, 250000, 500000, 3000000])

In [ ]:
df['income_cat'].value_counts()

In [ ]:
df['total_income'].describe()

In [ ]:
df['total_income'].quantile(q=0.25)

In [ ]:
df['income_cat'] = pd.cut(df['total_income'],[0, 100000, 150000, 200000, 3000000])
df['income_cat'].value_counts()

In [ ]:
df.pivot_table(index='income_cat',
              values=['debt'],
               aggfunc=['count','mean'])

## После более равномерной категоризации: у заемщиков с доходом до 100000 просрочка 8%, что на 0,5% меньше чем у заемщиков с доходом от 100000 до 200000. У заемщиков с доходом больше 200000 просрочка еще меньше, она составляет 7%.

### Вывод

## Категории заемщиков с самым маленьким и самым большим доходом более дисциплинированные плательщики по кредитам

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df['purpose_category'].value_counts()

In [ ]:
df.groupby('purpose_category')['debt'].agg(['count','mean'])

In [ ]:
df.groupby('purpose_category')['debt'].mean().plot()

### Вывод

## Просрочка выше по автокредитам и кредитам на образование (9%). Самая низкая просрочка по кредитам на недвижимость (7%), чуть выше по кредитам на свадьбу (8%).

In [ ]:
df.pivot_table(index='purpose_category',
              values=['debt'],
               aggfunc=['count','mean'])

### Шаг 4. Общий вывод

## В целом по кредитному портфелю процент просроченных кредитов небольшой, порядка 8%, что свидетельствует о довольно высоком качестве портфеля.
## У бездетных самая маленькая просрочка (7%), у клиентов с детьми незначительно выше процент просрочки (порядка 9,5%)
## У семейных людей меньше просрочек по кредитам, разница составляет около 1%, а также, у людей не состоявших в семейных отношениях на 2% больше просрочек
##  У заемщиков с доходом до 100000 просрочка 8%, что на 0,5% меньше чем у заемщиков с доходом от 100000 до 200000. У заемщиков с доходом больше 200000 просрочка еще меньше, она составляет 7%
## Просрочка выше по автокредитам и кредитам на образование (9%). Самая низкая просрочка по кредитам на недвижимость (7%), чуть выше по кредитам на свадьбу (8%)